### Importing Libraries

In [9]:
import numpy as np
%matplotlib inline
from pyqubo import Array, Placeholder, Constraint
import matplotlib.pyplot as plt
import networkx as nx
import neal
import csv
import os
import math

In [2]:
def euclidean_distance(x1, y1, x2, y2):
    return ((x1 - x2)**2 + (y1 - y2)**2) ** 0.5

def parse_tsp_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    node_coord_section = False
    node_matrix = []
    distance_matrix = None

    for line in lines:
        if line.startswith("NODE_COORD_SECTION"):
            node_coord_section = True
            continue
        elif line.startswith("EOF"):
            break

        if node_coord_section:
            node_info = line.split()
            node_id = int(node_info[0])
            # x_coord = int(float(node_info[1]))  # Convert to int
            # y_coord = int(float(node_info[2]))  # Convert to int
            x_coord = float(node_info[1])  # Convert to int
            y_coord = float(node_info[2])  # Convert to int

            node_matrix.append([node_id, x_coord, y_coord])

    node_matrix = np.array(node_matrix)

    # Calculate distance matrix (as in the previous version)
    num_nodes = node_matrix.shape[0]
    distance_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            x1, y1 = node_matrix[i, 1], node_matrix[i, 2]
            x2, y2 = node_matrix[j, 1], node_matrix[j, 2]
            distance_matrix[i, j] = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    return node_matrix, distance_matrix

In [3]:
def plot_city(cities, sol=None):
    n_city = len(cities)
    cities_dict = dict(cities)
    G = nx.Graph()
    for city in cities_dict:
        G.add_node(city)
        
    # draw path
    if sol:
        city_order = []
        for i in range(n_city):
            for j in range(n_city):
                if sol.array('c', (i, j)) == 1:
                    city_order.append(j)
        for i in range(n_city):
            city_index1 = city_order[i]
            city_index2 = city_order[(i+1) % n_city]
            G.add_edge(cities[city_index1][0], cities[city_index2][0])

    plt.figure(figsize=(3,3))
    pos = nx.spring_layout(G)
    nx.draw_networkx(G, cities_dict)
    plt.axis("off")
    plt.show()

def dist(i, j, cities):
    pos_i = cities[i][1]
    pos_j = cities[j][1]
    return np.sqrt((pos_i[0] - pos_j[0])**2 + (pos_i[1] - pos_j[1])**2)

In [17]:
def solve_for_instances(file_path, nodes_matrix, relaxation_parameter_start, relaxation_parameter_end, relaxation_parameter_step_size, instanceNumber=0, num_cities_in_a_instance=20):
    cities = [(i,(0,0)) for i in range(num_cities_in_a_instance)]
    n_mat = nodes_matrix.copy()
    for i in range(num_cities_in_a_instance):
        cities[i] = (str(n_mat[i][0]), (n_mat[i][1], n_mat[i][2]))
    #plot_city(cities)
   # print(cities)

    ## Prepare binary vector with bit (𝑖,𝑗) representing to visit 𝑗 city at time 𝑖
    n_city = len(cities)
    x = Array.create('c', (n_city, n_city), 'BINARY')
    # print(x)

    ## Constraint not to visit more than two cities at the same time.
    ## equation (6) implemented here
    ## time_const + city_const = H_a
    time_const = 0.0
    for i in range(n_city):
        # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
        time_const += Constraint((sum(x[i, j] for j in range(n_city)) - 1)**2, label="time{}".format(i))

    ## Constraint not to visit the same city more than twice.
    city_const = 0.0
    for j in range(n_city):
        city_const += Constraint((sum(x[i, j] for i in range(n_city)) - 1)**2, label="city{}".format(j))

    # print("time_const: ", time_const)
    # print("city_const: ", city_const)

    ## distance of route
    ## equation (5) implemented here
    distance = 0.0
    for i in range(n_city):
        for j in range(n_city):
            for k in range(n_city):
                d_ij = dist(i, j, cities)
                distance += d_ij * x[k, i] * x[(k+1)%n_city, j]  # sum(d_uv) in eq.5 i.e. H_b

    # print("distance: ", distance)

    ## Construct hamiltonian
    A = Placeholder("A")  # the relaxation parameter
    H = distance + A * (time_const + city_const)  # Eq (4)

    # print("Relaxation Parameter A: ", A)
    # print("Hamiltonian H: ", H)

    ## Compile model
    model = H.compile()

    ## Generate QUBO
    ## maybe we can add a loop to go through different relaxation parameter here
    feed_dict = {'A': 300.0}  # setting it to upper bound of the coordinate works!!!
    bqm = model.to_bqm(feed_dict=feed_dict)


    sa = neal.SimulatedAnnealingSampler()
    r_values = np.arange(relaxation_parameter_start, relaxation_parameter_end, relaxation_parameter_step_size)
    Pf_arr = []
    r_arr = []
    E_avg_arr = []
    E_std_arr = []
    energy_arr = []
    for r in r_values:
        feed_dict = {'A': r}   
        bqm = model.to_bqm(feed_dict=feed_dict)
        sampleset = sa.sample(bqm, num_reads=128, num_sweeps=128)

        decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
        best_sample = min(decoded_samples, key=lambda x: x.energy)
        energy = best_sample.energy
        num_broken = len(best_sample.constraints(only_broken=True))
        infeasible_ctr = 0
        for i in range(128):
            if len(decoded_samples[i].constraints(only_broken=True)) > 0:
                infeasible_ctr += 1
        P_f = (128-infeasible_ctr)/128
        print(f"P_f for {feed_dict} is {P_f}")


        energies = [0 for i in range(128)]
        for i in range(128):
            energies[i] = decoded_samples[i].energy
        E_avg = sum(energies)/128
        s = 0
        for i in range(128):
            s = s + (E_avg-energies[i])**2
        E_std = math.sqrt((s/128))
        E_avg_arr.append(E_avg)
        E_std_arr.append(E_std)
        energy_arr.append(energy)
        Pf_arr.append(P_f)
        r_arr.append(r)

    ## Define the filename for the plot
    '''result_plot_folder = "ResultPlots"
    plot_base_filename = os.path.basename(file_path)
    result_filename = os.path.splitext(plot_base_filename)[0]
    if not os.path.exists(result_plot_folder):
        os.makedirs(result_plot_folder)

    plot_filename = f"{result_plot_folder}/{result_filename}_instance_{instanceNumber}_plot.png"

    ## Plot the graph
    plt.scatter(r_arr,Pf_arr)
    plt.xlabel("Relaxation Parameter")
    plt.ylabel("$P_f$")
    plt.title(f"File: {file_path}\nInstance: {instanceNumber}")

    ## Save the plot as a PNG image in the 'ResultPlots' folder
    plt.savefig(plot_filename)

    plt.close()'''  # Close the current figure to avoid it being shown by plt.show()

    return r_arr, Pf_arr, E_avg_arr, E_std_arr, energy_arr  #returns relaxation parameter, Pf, min, mean and std of energies

In [24]:
def processTsp(file_path, relaxation_parameter_lower_bound, relaxation_parameter_upper_bound, num_plot_points, num_cities_in_a_instance=30):
    nodes_matrix, distance_matrix = parse_tsp_file(file_path)

    num_nodes = nodes_matrix.shape[0]
    num_instances = 1

    for instanceNumber in range(num_instances):
        start_index = instanceNumber * num_cities_in_a_instance
        end_index = min((instanceNumber + 1) * num_cities_in_a_instance, num_nodes)

        instance_nodes_matrix = nodes_matrix[start_index:end_index]
        instance_distance_matrix = distance_matrix[start_index:end_index, start_index:end_index]
        
        relaxation_parameter_start = relaxation_parameter_lower_bound
        relaxation_parameter_end = relaxation_parameter_upper_bound
        relaxation_parameter_step_size = (relaxation_parameter_upper_bound - relaxation_parameter_lower_bound) / num_plot_points

        if len(instance_nodes_matrix) == 0:
            break  # No more nodes to process

        r_arr, Pf_arr, E_avg, E_std, E_min = solve_for_instances(file_path, instance_nodes_matrix, relaxation_parameter_start, relaxation_parameter_end, relaxation_parameter_step_size, instanceNumber+1, len(instance_nodes_matrix))

        ## Save the result in a csvfile
        # Create a list of tuples (Relaxation Parameter A, Probability P_f)
        result_data = list(zip(r_arr, Pf_arr, E_avg, E_std, E_min))

        # Define the filename for the CSV file
        results_folder = "ResultsFolder"
        base_filename = os.path.basename(file_path)
        result_filename = os.path.splitext(base_filename)[0]
        csv_file = f"{results_folder}/{result_filename}_instance{instanceNumber+1}.csv"

        # Write the data to the CSV file
        with open(csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Relaxation Parameter A', 'Probability P_f', 'E_avg', "E_std", "E_min"])
            writer.writerows(result_data)

        print(f"Results for instance {base_filename} saved to {csv_file}.")

In [25]:
def process_tsp_files_in_folder(folder_name, parameter_boundary_file_path, num_plot_points, num_cities_in_a_instance=30):
    if not os.path.exists(folder_name):
        raise ValueError(f"Folder '{folder_name}' does not exist.")

    # Read Parameter_range.csv file and store it into array of dictionaries  of type { File_name: [Upper_bound, Lower_bound]}
    boundary_list = {}

    with open(parameter_boundary_file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            # Convert Upper_bound and Lower_Bound values to integers
            upper_bound = int(row['Upper_bound'])
            lower_bound = int(row['Lower_Bound'])
            
            # Populate the dictionary with File_name as key and [lower_bound, upper_bound] as value
            boundary_list[row['File_name']] = [lower_bound, upper_bound]

    # Process TSP Files
    files_in_folder = [file for file in os.listdir(folder_name) if os.path.isfile(os.path.join(folder_name, file))]

    for file_name in files_in_folder:
        if file_name.endswith('.tsp'):
            file_path = os.path.join(folder_name, file_name)
            print(f"Processing file: {file_name}")
            relaxation_parameter_lower_bound = boundary_list[file_name][0]
            relaxation_parameter_upper_bound = boundary_list[file_name][1]
            processTsp(file_path, relaxation_parameter_lower_bound, relaxation_parameter_upper_bound, num_plot_points, num_cities_in_a_instance)
            print("----------------------------------------------------")

In [26]:
folder_name = "TSPFiles"
parameter_boundary_file_path = "ParameterBoundaryResults/parameter_range.csv"
num_plot_points = 100
# relaxation_parameter_start = 20
# relaxation_parameter_end = 80
# relaxation_parameter_step_size = 2
process_tsp_files_in_folder(folder_name, parameter_boundary_file_path, num_plot_points, num_cities_in_a_instance=30)

Processing file: pr1002.tsp
P_f for {'A': 2187.0} is 0.0234375
P_f for {'A': 2208.88} is 0.03125
P_f for {'A': 2230.76} is 0.0390625
P_f for {'A': 2252.6400000000003} is 0.0625
P_f for {'A': 2274.5200000000004} is 0.0703125
P_f for {'A': 2296.4000000000005} is 0.0625
P_f for {'A': 2318.2800000000007} is 0.0625
P_f for {'A': 2340.1600000000008} is 0.109375
P_f for {'A': 2362.040000000001} is 0.140625
P_f for {'A': 2383.920000000001} is 0.078125
P_f for {'A': 2405.800000000001} is 0.078125
P_f for {'A': 2427.680000000001} is 0.1328125
P_f for {'A': 2449.5600000000013} is 0.1171875
P_f for {'A': 2471.4400000000014} is 0.2421875
P_f for {'A': 2493.3200000000015} is 0.1796875
P_f for {'A': 2515.2000000000016} is 0.109375
P_f for {'A': 2537.0800000000017} is 0.203125
P_f for {'A': 2558.960000000002} is 0.2109375
P_f for {'A': 2580.840000000002} is 0.21875
P_f for {'A': 2602.720000000002} is 0.234375
P_f for {'A': 2624.600000000002} is 0.2578125
P_f for {'A': 2646.4800000000023} is 0.2421875
